In [2]:
import pandas as pd
import numpy as np
from settings import Config
from mysql_db import Database
import pdcast as pdc
import s3_upload_download as s3con
import os
from datetime import datetime

# NOTEBOOK DESCRIPTION: 

Investigate the payments, the rezerves and ensure that every event has a monetary value (payment + rezerves) associated with it.


In [5]:
# initialise the s3_connector object needed to read/write files into an S3 bucket
s3con = s3con.s3_connector()

In [3]:
df = pd.read_feather('plati_eq_grupat.feather')
df.head()

,dosarID,ron_eq,eur_eq
0,53151,51041.0,11349.253623
1,53156,7759.0,1722.952532
2,53160,43530.0,9584.089587
3,53171,25667.0,5730.342896
4,53177,5663.0,1267.908959


In [7]:
df.shape[0]

761531

In [4]:
df1 = pd.read_feather('plati_eq_wFinale.feather')
df1.head()

,dosarID,ron_eq,eur_eq,plata_finala
0,53151,84604.0,19148.108732,yes
1,53156,7759.0,1721.238703,yes
2,53160,21765.0,4869.127745,yes
3,53171,25667.0,5754.416509,yes
4,53177,5663.0,1273.041987,yes


In [16]:
df1.shape[0]

627281

In [6]:
df3 = s3con.read('daune.feather')
df3.head()

,idDosar,idPolita,stare,dataDeschidere,dataAvizare,dataEveniment,tipEveniment,tipDauna,idEvent
0,59897,2231006,inchis - achitat,2015-01-06,2015-01-06,2015-01-05,Pagube materiale,INTERNA,22310062015-01-05
1,60564,2233977,inchis - achitat,2015-01-12,2015-01-12,2015-01-07,Pagube materiale,INTERNA,22339772015-01-07
2,60672,2231966,inchis - achitat,2015-01-13,2015-01-13,2015-01-09,Pagube materiale,INTERNA,22319662015-01-09
3,60732,2232835,inchis - achitat,2015-01-13,2015-01-13,2015-01-08,Pagube materiale,INTERNA,22328352015-01-08
4,60984,2239842,inchis - achitat,2015-01-14,2015-01-14,2015-01-09,Pagube materiale,INTERNA,22398422015-01-09


In [15]:
df3.idEvent.nunique()

638826

In [18]:
df3.duplicated('idEvent').sum()

49116